In [ ]:
# Rule 1-36
folder_paths = [
        ".\Mix_G_5",
    ]
    C = [
        "_1",
        "_2",
        "_3",
        "_4",
        "_5",
        "_6",
        "_over6",
    ]
!sourcery review - -config.sourcery.yaml
for folder_path in folder_paths:
    for c in C:
        new_folder_path = os.path.join(folder_path, folder_path[2:] + c)
        !sourcery review - -fix {new_folder_path}
        python_files = [f for f in os.listdir(new_folder_path) if f.endswith('.py')]

# Rule 37-40
import ast
import hashlib
import astor

def get_sorted_hash(value):
    sorted_value = ''.join(sorted(value))
    return hashlib.sha256(sorted_value.encode()).hexdigest()


def transform_operations(node, lines, file_path):
    if isinstance(node, ast.BinOp):
        if isinstance(node.op, ast.Add):
            left_var = node.left.id if isinstance(node.left, ast.Name) else None
            right_var = node.right.id if isinstance(node.right, ast.Name) else None

            if left_var and right_var:
                hash_left = get_sorted_hash(left_var)
                hash_right = get_sorted_hash(right_var)

                original_code = astor.to_source(node)

                print(f"Comparing hashes: {left_var}: {hash_left}, {right_var}: {hash_right}")

                if hash_left > hash_right:
                    node.left, node.right = node.right, node.left

                    transformed_code = astor.to_source(node).strip()

                    if original_code.strip() != transformed_code:
                        transformed_line = lines[node.lineno - 1].replace(original_code.strip(), transformed_code)
                        lines[node.lineno - 1] = transformed_line
                        print(f"Transformed in file: {file_path}")
                        print(f"Original line: {lines[node.lineno - 1].strip()}")
                        print(f"Transformed line: {transformed_code.strip()}\n")
                        return True
    return False

def process_file(file_path):
    with open(file_path, 'r', encoding='UTF-8') as file:
        content = file.read()
        lines = content.split('\n')

    tree = ast.parse(content)
    modified = False

    for node in ast.walk(tree):
        if transform_operations(node, lines, file_path):
            modified = True

    if modified:
        new_content = astor.to_source(tree)

        with open(file_path, 'w', encoding='UTF-8') as file:
            new_content = '\n'.join(line for line in new_content.split('\n') if line.strip())
            file.write(new_content)

            os.remove(file_path)
            print(f"Original file {file_path} deleted.")
        print(f"File transformed and saved as {file_path}")


def process_files(file_path):
                try:
                    process_file(file_path)
                except Exception as e:
                    print(f"Error processing file {file_path}: {e}")

if __name__ == "__main__":
    file_path = "./Mix_G_5/**.py"

    process_files(file_path)

# Rule 41-47
import os
import autopep8

def process_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()

    formatted_code = autopep8.fix_code(content, options={'ignore': ['E101,E11,E121,E122,E125']})
    file_changed = False

    if content != formatted_code:
        print('Modified code')
    else:
        print('Same as the original code')

    with open(file_path, 'w') as file:
        file.write(formatted_code)

    if file_changed:
        os.remove(file_path)
        print(f"Original file {file_path} deleted.")

    print(f"File transformed and saved as {file_path}")

def process_files(file_path):

                try:
                    process_file(file_path)
                except Exception as e:
                    print(f"Error processing file {file_path}: {e}")

if __name__ == "__main__":
    file_path = "./Mix_G_5/**.py"

    process_files(file_path)